In [ ]:
import numpy

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to images and masks directories
image_dir = "C:/Users/Welcome/Downloads/Banana FCN/Images"
mask_dir = "C:/Users/Welcome/Downloads/Banana FCN/Mask"

In [ ]:
image_datagen = ImageDataGenerator(rescale=1./255)
mask_datagen = ImageDataGenerator(rescale=1./255)

image_generator = image_datagen.flow_from_directory(
    image_dir,
    class_mode=None,
    color_mode='rgb',
    target_size=(128, 128),
    batch_size=32,
    seed=42
)

mask_generator = mask_datagen.flow_from_directory(
    mask_dir,
    class_mode=None,
    color_mode='grayscale',
    target_size=(128, 128),
    batch_size=32,
    seed=42
)

In [ ]:
train_generator = zip(image_generator, mask_generator)


from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, Input
from tensorflow.keras.models import Model

In [ ]:
def build_fcnn():
    inputs = Input((128, 128, 3))

    # Encoder
    conv1 = Conv2D(128, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D((2, 2))(conv2)

    # Decoder
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    up1 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv3)

    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(up1)
    up2 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv4)

    outputs = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(up2)

    model = Model(inputs, outputs)
    return model

In [ ]:
model = build_fcnn()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train the FCNN model
def combined_generator(image_gen, mask_gen):
    while True:  # Keep yielding data indefinitely
        img_batch = next(image_gen)
        mask_batch = next(mask_gen)
        yield img_batch, mask_batch  # Keras expects (input, target)
# Fit the model with the custom generator
train_generator = combined_generator(image_generator, mask_generator)
model.fit(train_generator, steps_per_epoch=len(image_generator), epochs=50)

import matplotlib.pyplot as plt
import numpy as np

# Sample image for prediction
sample_image = image_generator[0][0]
predicted_mask = model.predict(np.expand_dims(sample_image, axis=0))[0]

# Display the original image and predicted mask
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(sample_image)

plt.subplot(1, 2, 2)
plt.title("Predicted Mask")
plt.imshow(predicted_mask.squeeze(), cmap='gray')

plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to images and masks directories
image_dir = "C:/Users/Welcome/Downloads/Banana FCN/Images/Images"
mask_dir = "C:/Users/Welcome/Downloads/Banana FCN/Mask/Mask"

# Image and mask data generators
image_datagen = ImageDataGenerator(rescale=1./255)
mask_datagen = ImageDataGenerator(rescale=1./255)

image_generator = image_datagen.flow_from_directory(
    image_dir,
    class_mode=None,
    color_mode='rgb',
    target_size=(128, 128),
    batch_size=32,
    seed=42
)

mask_generator = mask_datagen.flow_from_directory(
    mask_dir,
    class_mode=None,
    color_mode='grayscale',
    target_size=(128, 128),
    batch_size=32,
    seed=42
)

# Combine generators into one which yields image and mask
train_generator = zip(image_generator, mask_generator)


from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, Input
from tensorflow.keras.models import Model

def build_fcnn():
    inputs = Input((128, 128, 3))

    # Encoder
    conv1 = Conv2D(128, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D((2, 2))(conv2)

    # Decoder
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    up1 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv3)

    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(up1)
    up2 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv4)

    outputs = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(up2)

    model = Model(inputs, outputs)
    return model

model = build_fcnn()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train the FCNN model
def combined_generator(image_gen, mask_gen):
    while True:  # Keep yielding data indefinitely
        img_batch = next(image_gen)
        mask_batch = next(mask_gen)
        yield img_batch, mask_batch  # Keras expects (input, target)
# Fit the model with the custom generator
train_generator = combined_generator(image_generator, mask_generator)
model.fit(train_generator, steps_per_epoch=len(image_generator), epochs=50)

import matplotlib.pyplot as plt
import numpy as np

# Sample image for prediction
sample_image = image_generator[0][0]
predicted_mask = model.predict(np.expand_dims(sample_image, axis=0))[0]

# Display the original image and predicted mask
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(sample_image)

plt.subplot(1, 2, 2)
plt.title("Predicted Mask")
plt.imshow(predicted_mask.squeeze(), cmap='gray')

plt.show()